# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

#Configure Gmaps
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather = "../WeatherPy/Output/city_weather_data.csv"

#pull in csv and drop odd index row
weather_data = pd.read_csv(weather,  index_col="Unnamed: 0")

# Shows data frame and sets indel to city
weather_data.head(10).set_index("City")

,Country,Latitude,Longitude,Temperature High,Temperature Low,Feels Like Temp.,Humidity,Cloudiness,Wind Speed
City,,,,,,,,,
Omboué,GA,-1.5746,9.2618,71.24,71.24,72.16,87,37,6.22
Busselton,AU,-33.6500,115.3333,56.82,56.82,56.30,87,84,16.02
Manavālakurichi,IN,8.1333,77.3000,84.29,84.29,90.45,68,85,17.43
Mataura,NZ,-46.1927,168.8643,45.48,45.48,45.48,86,100,1.68
Upernavik,GL,72.7868,-56.1549,42.26,42.26,37.42,83,100,7.85
Kapaa,US,22.0752,-159.3190,85.98,76.19,78.93,76,40,5.01
Atkinson,US,42.8384,-71.1470,74.43,64.00,69.04,88,1,5.99
Leningradskiy,RU,69.3833,178.4167,38.16,38.16,31.80,70,27,9.04
Evensk,RU,61.9500,159.2333,53.24,53.24,52.16,83,42,8.59


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Store 'Lat' and 'Lng' into  locations 
locations = weather_data[["Latitude", "Longitude"]].astype(float)

# sets weight as humidity for a coordinate
weight = weather_data["Humidity"].astype(float)

# Variable for heat map intensity
max_humidity = weather_data["Humidity"].max()


In [4]:
# Create a Humidity Heatmap layer
humidity_heatmap = gmaps.figure(zoom_level=2, center=(0,0),layout={
        'width': '600px',
        'height': '600px',
        'padding': '3px',
        'border': '2px solid black'})

# Adds a humidity layer to the heatmap and applies intensity from above
humidity_layer = gmaps.heatmap_layer(locations, weights=weight, 
                                 dissipating=False, max_intensity=max_humidity,
                                 point_radius = 3)

# Adds layer created in line above to the map
humidity_heatmap.add_layer(humidity_layer)

#Shows the humidity heat map
humidity_heatmap

Figure(layout=FigureLayout(border='2px solid black', height='600px', padding='3px', width='600px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [8]:
# Finds perfect conditions. Temp between 60 and 80, less than 50% humidity, and windspeed higher than 6mph
perfect_conditions_not_cleaned = weather_data[(weather_data["Temperature High"] > 60) &
                                  (weather_data["Temperature High"] < 80) &
                                  (weather_data["Humidity"] < 50) &
                                  (weather_data["Wind Speed"] > 6)&
                                  (weather_data["Cloudiness"]< 40) &
                                  (weather_data["Feels Like Temp."]< 75)
                                   ]
# Dropping all na values
perfect_conditions = perfect_conditions_not_cleaned.dropna(axis=0, how='any')
perfect_conditions

,City,Country,Latitude,Longitude,Temperature High,Temperature Low,Feels Like Temp.,Humidity,Cloudiness,Wind Speed
36,Tuktoyaktuk,CA,69.4541,-133.0374,67.68,67.68,66.34,47,8,13.98
42,Mount Isa,AU,-20.7333,139.5000,74.97,74.97,73.24,23,13,10.36
53,Yeppoon,AU,-23.1333,150.7333,69.58,69.58,67.89,35,1,14.20
105,Ballina,AU,-28.8667,153.5667,62.78,62.78,60.78,43,16,10.76
240,Alice Springs,AU,-23.7000,133.8833,74.21,72.95,72.32,21,1,11.99
261,Byron Bay,AU,-28.6500,153.6167,64.35,61.88,60.87,43,7,12.68
319,Yulara,AU,-25.2406,130.9889,75.04,75.04,73.33,23,5,9.22
438,Emerald,AU,-23.5333,148.1667,70.34,68.14,65.79,24,1,17.27
458,Ust'-Kulom,RU,61.6864,53.6902,66.07,66.07,64.60,47,6,6.49
659,João Pinheiro,BR,-17.7425,-46.1725,60.62,60.62,57.42,22,5,6.42


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [10]:
# Create a hotel_df
hotel_df = perfect_conditions.loc[:,["City","Country", "Latitude", "Longitude","Cloudiness"]]

# Add a "Hotel Name" column to the hotel_df DataFrame.
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Latitude,Longitude,Cloudiness,Hotel Name
36,Tuktoyaktuk,CA,69.4541,-133.0374,8,
42,Mount Isa,AU,-20.7333,139.5000,13,
53,Yeppoon,AU,-23.1333,150.7333,1,
105,Ballina,AU,-28.8667,153.5667,16,
240,Alice Springs,AU,-23.7000,133.8833,1,
261,Byron Bay,AU,-28.6500,153.6167,7,
319,Yulara,AU,-25.2406,130.9889,5,
438,Emerald,AU,-23.5333,148.1667,1,
458,Ust'-Kulom,RU,61.6864,53.6902,6,
659,João Pinheiro,BR,-17.7425,-46.1725,5,


In [11]:
# Creating parameters to pass in to api url
parameters = {
    "keyword": "Hotel",
    "radius": 5000,
    "type": "Lodging",
    "key": g_key}

# Defining base url for query of API
base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json?'

# For loop to go through hotel_df and extract data for search, will keep count of index too
for index, place in hotel_df.iterrows():
        
        # Creating variables first, calls the city,lat,and lng of each row in hotel_df
        city = place["City"]
        lat = place["Latitude"]
        lng = place["Longitude"]
        #creating new parameter based on lat/lng created above
        parameters["location"] = f'{lat},{lng}'
        print('------------------------------------------------------------------------------')
        print()
        print()
        print(f'Getting data for {city}, City #:{index}')
        print()
        print('Accessing API, please wait..')
         
        #Creating url with parameters above
        response = requests.get(base_url, params=parameters).json()
        # Saving json results to hotel_name
        hotel_name = response["results"]
        try:
            print()
            print(f"The closest hotel to {city}'s coordinates is: {hotel_name[2]['name']}")
            print()
            print()
            print('------------------------------------------------------------------------------')
            
            # Finds Hotel Name column, then locates the current index in .iterrows() to add "Hotel Name" to the correct City
            hotel_df.loc[index, "Hotel Name"] = hotel_name[2]['name']
        # If there is a key error or index error, return below lines
        except(KeyError, IndexError) as error:
            print(f'Unable to find nearest hotel in{city}. Error: {error}')
            print('------------------------------------------------')
print()
print()
print("Finished collecting nearest hotels")

------------------------------------------------------------------------------


Getting data for Tuktoyaktuk, City #:36

Accessing API, please wait..

The closest hotel to Tuktoyaktuk's coordinates is: Tuktu B&B


------------------------------------------------------------------------------
------------------------------------------------------------------------------


Getting data for Mount Isa, City #:42

Accessing API, please wait..

The closest hotel to Mount Isa's coordinates is: Outback Motel Mt Isa


------------------------------------------------------------------------------
------------------------------------------------------------------------------


Getting data for Yeppoon, City #:53

Accessing API, please wait..

The closest hotel to Yeppoon's coordinates is: Yeppoon Beach House


------------------------------------------------------------------------------
------------------------------------------------------------------------------


Getting data for Ballina, Ci

In [12]:
# Replaces empty cells in "Hotel Name" with Nan
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
# Drops rows with Nan in "Hotel Name"
hotel_df.dropna(subset=['Hotel Name'], inplace=True)
hotel_df

,City,Country,Latitude,Longitude,Cloudiness,Hotel Name
36,Tuktoyaktuk,CA,69.4541,-133.0374,8,Tuktu B&B
42,Mount Isa,AU,-20.7333,139.5000,13,Outback Motel Mt Isa
53,Yeppoon,AU,-23.1333,150.7333,1,Yeppoon Beach House
105,Ballina,AU,-28.8667,153.5667,16,Slipway Hotel - Motel
240,Alice Springs,AU,-23.7000,133.8833,1,Alice Springs Airport Motel
261,Byron Bay,AU,-28.6500,153.6167,7,The Beach Shack
319,Yulara,AU,-25.2406,130.9889,5,Sails in the Desert
438,Emerald,AU,-23.5333,148.1667,1,Direct Hotels - Villas on Rivergum
659,João Pinheiro,BR,-17.7425,-46.1725,5,Hotel Avenida


In [13]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [14]:
city_names = list(hotel_df["City"])
# Add marker layer ontop of heat map, adding hover detail
humidity_heatmap.add_layer(gmaps.marker_layer(hotel_df[["Latitude","Longitude"]],
                  hover_text=city_names))

# Display figure with overlay of markers
humidity_heatmap

Figure(layout=FigureLayout(border='2px solid black', height='600px', padding='3px', width='600px'))